# Querying the dynamical.org Static STAC Catalog

This notebook shows how to query the [dynamical.org STAC catalog](https://opensciencecomputing.github.io/dynamical-stac/stac/dynamical/catalog.json)
hosted on GitHub Pages.

The catalog is a **static** STAC catalog (plain JSON files), so queries are performed
client-side using `pystac` and `shapely`. For larger catalogs a STAC API server would
be needed for efficient server-side filtering.

## Install dependencies
```
pip install pystac shapely
```

In [ ]:
import pystac
from shapely.geometry import box, shape

CATALOG_URL = (
    "https://opensciencecomputing.github.io/dynamical-stac"
    "/stac/dynamical/catalog.json"
)

catalog = pystac.Catalog.from_file(CATALOG_URL)
print(catalog.description)
print(f"\n{len(list(catalog.get_items()))} items in catalog")

## 1. List all items

In [ ]:
for item in catalog.get_items():
    print(f"{item.id}")
    print(f"  title:      {item.common_metadata.title}")
    print(f"  bbox:       {item.bbox}")
    print(f"  start:      {item.common_metadata.start_datetime}")
    print(f"  end:        {item.common_metadata.end_datetime}")
    print()

## 2. Spatial query — find datasets that cover a region

Use `shapely` to test geometry intersection against a bounding box of interest.
Here we query for datasets covering Italy.

In [ ]:
# Define region of interest (lon_min, lat_min, lon_max, lat_max)
roi = box(6.6, 36.6, 18.5, 47.1)  # Italy

matches = [
    item for item in catalog.get_items()
    if shape(item.geometry).intersects(roi)
]

print(f"Datasets covering Italy ({len(matches)} of {len(list(catalog.get_items()))}):\n")
for item in matches:
    print(f"  {item.id}")

## 3. Temporal query — find datasets covering a date range

In [ ]:
from datetime import datetime, timezone

query_start = datetime(2024, 1, 1, tzinfo=timezone.utc)
query_end   = datetime(2025, 1, 1, tzinfo=timezone.utc)

def overlaps_time(item, start, end):
    item_start = item.common_metadata.start_datetime
    item_end   = item.common_metadata.end_datetime
    if item_start is None or item_end is None:
        return True  # no temporal info — include by default
    return item_start <= end and item_end >= start

matches = [
    item for item in catalog.get_items()
    if overlaps_time(item, query_start, query_end)
]

print(f"Datasets covering {query_start.date()} – {query_end.date()} ({len(matches)}):\n")
for item in matches:
    print(f"  {item.id}")
    print(f"    {item.common_metadata.start_datetime} → {item.common_metadata.end_datetime}")

## 4. Combined spatial + temporal query

In [ ]:
roi         = box(6.6, 36.6, 18.5, 47.1)   # Italy
query_start = datetime(2024, 1, 1, tzinfo=timezone.utc)
query_end   = datetime(2025, 1, 1, tzinfo=timezone.utc)

matches = [
    item for item in catalog.get_items()
    if shape(item.geometry).intersects(roi)
    and overlaps_time(item, query_start, query_end)
]

print(f"Datasets covering Italy and {query_start.date()} – {query_end.date()} ({len(matches)}):\n")
for item in matches:
    print(f"  {item.id}")

## 5. Open a matching dataset with xarray

Pick the first match and open it directly via `xpystac`.

In [ ]:
import xarray as xr
import xpystac  # registers xarray backend for icechunk assets

item = matches[0]
print(f"Opening: {item.id}")

asset_key = next(k for k in item.assets if "@" in k)
asset = item.assets[asset_key]

ds = xr.open_dataset(asset)
ds